<a href="https://colab.research.google.com/github/blid11/Group9/blob/Brett/Copy_of_G9_PartC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Changes made in this Colab notebook can be saved with Control/Command S and do not need to be pushed to Github. 

In [4]:

# Import libaries needed
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

# made a change in colab

In [1]:
from google.colab import drive 
drive.mount('/content/gdrive', force_remount=True)

%cd gdrive/My Drive/G9A2/G9
 
#%cd gdrive/My Drive/G9A2
# I made a folder on my google drive that has the G9 Methylation data and the binfpy library
#this code automatically loads it in
# you can do the same with your google drive. 

Mounted at /content/gdrive
/content/gdrive/My Drive/G9A2/G9


In [ ]:
#set styles of plots - taken from Week 6 notebook
sns.set_theme(style="ticks")

sns.set(rc={'figure.figsize': (5, 5), 'font.family': 'sans-serif',
            'font.sans-serif': 'Arial', 'font.size': 10, 'axes.titlesize': 16,
           'axes.titleweight': 700}, style='ticks')
sns.set_palette("Set2")

# Data Preparation


In [2]:
!ls

G9_liver_dna-meth.csv  G9_liver_gene-expr.csv


In [7]:
# load the data 
liver_meth_dna = pd.read_csv('G9_liver_dna-meth.csv')
liver_gene_expr = pd.read_csv('G9_liver_gene-expr.csv')


In [9]:
# view the data: 
liver_meth_dna.head(n = 5)

,Unnamed: 0,Label,cg00005740,cg00012529,cg00013196,cg00019366,cg00028598,cg00030627,cg00033551,cg00047815,...,rs7660805,rs7746156,rs798149,rs845016,rs877309,rs9292570,rs9363764,rs939290,rs951295,rs966367
0,TCGA.2V.A95S.01,Primary Tumor,0.601838,0.374335,0.300235,0.259656,0.311692,0.331413,0.963102,0.586768,...,0.944546,0.959710,0.969422,0.368036,0.513791,0.509061,0.541551,0.562851,0.495167,0.048576
1,TCGA.2Y.A9GS.01,Primary Tumor,0.399472,0.395836,0.395846,0.380457,0.273681,0.446539,0.952035,0.380159,...,0.334419,0.459250,0.971834,0.933756,0.521390,0.020679,0.573997,0.960284,0.037918,0.473765
2,TCGA.2Y.A9GT.01,Primary Tumor,0.438239,0.640740,0.477210,0.519875,0.377109,0.450508,0.604261,0.558608,...,0.511502,0.462957,0.018341,0.951748,0.544744,0.025068,0.558837,0.966630,0.968370,0.948648
3,TCGA.2Y.A9GU.01,Primary Tumor,0.913263,0.969171,0.944687,0.573213,0.944506,0.934732,0.909746,0.883221,...,0.540377,0.487190,0.015734,0.945182,0.020356,0.521952,0.939739,0.966117,0.027995,0.949593
4,TCGA.2Y.A9GV.01,Primary Tumor,0.648241,0.805019,0.560080,0.914805,0.389027,0.641507,0.218462,0.559733,...,0.948858,0.957963,0.016105,0.940811,0.569243,0.967371,0.518856,0.523212,0.506449,0.037212


In [10]:
liver_gene_expr.head(n= 5)
#the first column is sample
#the second column is the Label 
# the third column onwards are Data (expression values for given gene)

,Unnamed: 0,Label,X..100130426,X..100133144,X..100134869,X..10357,X..10431,X..136542,X..155060,X..26823,...,ZXDA.7789,ZXDB.158586,ZXDC.79364,ZYG11A.440590,ZYG11B.79699,ZYX.7791,ZZEF1.23140,ZZZ3.26009,psiTPTE22.387590,tAKR.389932
0,TCGA.2V.A95S.01,Primary Tumor,0.0,1.5051,3.7074,90.1124,1017.1038,0.0,141.3911,0.6516,...,24.7597,273.6602,794.2662,18.2440,499.1041,3172.5037,890.0472,510.1808,3.9094,6.5157
1,TCGA.2Y.A9GS.01,Primary Tumor,0.0,26.4120,2.6663,71.0054,639.2311,0.0,122.7206,1.4786,...,68.5067,632.8241,1153.7703,71.4638,1000.4929,5301.1336,755.5446,860.5224,6.4071,482.9966
2,TCGA.2Y.A9GT.01,Primary Tumor,0.0,0.0000,4.4833,95.5122,742.4344,0.0,95.0460,1.7933,...,46.6263,1219.4575,1133.3782,12.5532,1289.3970,3219.0092,860.7935,523.6494,14.3466,83.3894
3,TCGA.2Y.A9GU.01,Primary Tumor,0.0,5.7222,5.1216,61.6679,1186.9807,0.0,280.2709,0.8341,...,18.3511,285.2758,1150.2786,9.1755,941.7437,3092.9899,1339.6283,343.6655,2.5024,2.5024
4,TCGA.2Y.A9GV.01,Primary Tumor,0.0,11.4975,5.4230,104.4670,878.1726,0.0,282.5719,0.0000,...,41.4552,999.1540,1631.9797,4.2301,1380.7107,2902.7073,575.2961,665.8206,2.5381,119.2893


In [16]:
# convert labels to 1 or 0 
bool_tum_expr = [1 if value == 'Primary Tumor' else 0 for value in liver_gene_expr['Label'].values]
bool_tum_meth = [1 if value == 'Primary Tumor' else 0 for value in liver_meth_dna['Label'].values]

# add the numeric label values to the dataframes
liver_gene_expr['BinaryLabel'] = bool_tum_expr
liver_meth_dna['BinaryLabel'] = bool_tum_meth

# I'm not sure if we need to convert the gene expression/methylation to numeric

In [17]:
#confirm that the changes were added
liver_gene_expr.head(n= 2)
liver_meth_dna.head(n = 2)

,Unnamed: 0,Label,cg00005740,cg00012529,cg00013196,cg00019366,cg00028598,cg00030627,cg00033551,cg00047815,...,rs7746156,rs798149,rs845016,rs877309,rs9292570,rs9363764,rs939290,rs951295,rs966367,BinaryLabel
0,TCGA.2V.A95S.01,Primary Tumor,0.601838,0.374335,0.300235,0.259656,0.311692,0.331413,0.963102,0.586768,...,0.95971,0.969422,0.368036,0.513791,0.509061,0.541551,0.562851,0.495167,0.048576,1
1,TCGA.2Y.A9GS.01,Primary Tumor,0.399472,0.395836,0.395846,0.380457,0.273681,0.446539,0.952035,0.380159,...,0.45925,0.971834,0.933756,0.521390,0.020679,0.573997,0.960284,0.037918,0.473765,1


will be creating two models: one for gene expression and one for methylation
so will not merge the two datasets 
we can use the same model architecture and train it on different data

first can standardize/normalize the data 
Then can log transform it 
then can set the values between 0 and 1?

# TensorBoard for Model Evaluation

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard
# %reload_ext tensorboard

%tensorboard --logdir /dir/

# or use %tensorboard --logdir /dir/

# Solid Tissue Normal vs Primary Tumour Classifier


In [ ]:
# Need to make data and labels variables 

import tensorflow as tf

np.random.seed(129)
tf.random.set_seed(129)

# JUST reference info below here

# Support Vector Machine

In [ ]:
## Encode the labels in 1 hot encoding
from sklearn import svm
from sklearn.metrics import plot_confusion_matrix
from sklearn.preprocessing import LabelEncoder
from keras.utils.np_utils import to_categorical

encoder = LabelEncoder()
encoder.fit(labels)
encoded_Y = encoder.transform(labels)
# convert integers to dummy variables (i.e. one hot encoded)
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.33, random_state=42)

# Run a SVM
# https://scikit-learn.org/stable/modules/svm.html
clf = svm.SVC(kernel='rbf')
clf.fit(X_train, y_train)
plot_confusion_matrix(clf, X_test, y_test)  
plt.show()
plot_confusion_matrix(clf, data, labels) 

# Neural Network


In [ ]:
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)
X_train, X_test, y_train, y_test = train_test_split(data, dummy_y, test_size=0.33, random_state=42)
# create model

model = Sequential()
model.add(Dense(2,  activation='sigmoid', input_shape=(3,),  name="bottleneck"))
model.add(Dense(3,  activation='softmax'))


#model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train, y_train, epochs = 150, batch_size = 4)

In [ ]:
scores = model.evaluate(X_test, y_test)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

In [ ]:
encoder_p = Model(model.input, model.get_layer('bottleneck').output)

nn_enc = encoder_p.predict(data)  # bottleneck representation

# plot
for tissue in tissues:
    tissue_idxs = np.where(df['tissue'].values == tissue)
    sc = plt.scatter(nn_enc[tissue_idxs, 0], nn_enc[tissue_idxs, 1], s=60, label=tissue, marker='o', alpha=0.8)
    
ax.set_xlabel('AE 0')
ax.set_ylabel('AE 1')
plt.legend()
plt.title('AE of age, years smoked, BMI')
plt.show()


df['AE0'] = nn_enc[:, 0]
df['AE1'] = nn_enc[:, 1]

# plot each of the features on the plot! We'll relate these later 
cmaps = ['BuGn', 'PuRd', 'YlOrBr']
for i, s in enumerate(['bmi', 'smoke', 'age']):    
    sc = plt.scatter(df['AE0'], df['AE1'], s=60, label=s, c=df[s], marker='o', alpha=0.8, cmap=cmaps[i])

    ax.set_xlabel('BMI')
    ax.set_ylabel('Smoked years')
    plt.colorbar()
    plt.title(f'AE BMI, Smoked years and Age, colour: {s}')
    plt.show()

# Check Metrics of Models

In [ ]:
from sklearn.metrics import confusion_matrix
from keras.utils.np_utils import to_categorical

predictions = model.predict(data)
prediction_ = np.argmax(predictions, axis = 1)
prediction_ = encoder.inverse_transform(prediction_)

for i, j in zip(prediction_ , labels):
    print( " the nn predict {}, and the tumour stage is {}".format(i,j))
confusion_matrix(prediction_, labels)


# New Section